In [1]:
# imports
import pandas as pd
# !pip install catboost

In [2]:
raw_df = pd.read_csv('flavors_of_cacao.csv', na_values=['\xa0'])

In [3]:
# raw_df.info()
# raw_df.describe()
raw_df.head()

,Company (Maker-if known),Specific Bean Origin or Bar Name,REF,Review Date,Cocoa Percent,Company Location,Rating,Bean Type,Broad Bean Origin
0,A. Morin,Agua Grande,1876,2016,63%,France,3.75,NaN,Sao Tome
1,A. Morin,Kpime,1676,2015,70%,France,2.75,NaN,Togo
2,A. Morin,Atsane,1676,2015,70%,France,3.00,NaN,Togo
3,A. Morin,Akata,1680,2015,70%,France,3.50,NaN,Togo
4,A. Morin,Quilla,1704,2015,70%,France,3.50,NaN,Peru


In [4]:
raw_df.isnull().sum()

Company \n(Maker-if known)             0
Specific Bean Origin\nor Bar Name      0
REF                                    0
Review\nDate                           0
Cocoa\nPercent                         0
Company\nLocation                      0
Rating                                 0
Bean\nType                           888
Broad Bean\nOrigin                    74
dtype: int64

In [5]:
df = raw_df.dropna(subset= ['Broad Bean\nOrigin'])

In [6]:
df.isnull().sum()

Company \n(Maker-if known)             0
Specific Bean Origin\nor Bar Name      0
REF                                    0
Review\nDate                           0
Cocoa\nPercent                         0
Company\nLocation                      0
Rating                                 0
Bean\nType                           837
Broad Bean\nOrigin                     0
dtype: int64

In [7]:
df.columns = [column.replace('\n','_') for column in df.columns]
df.columns = [column.replace('\xa0','') for column in df.columns]
df.columns

Index(['Company_(Maker-if known)', 'Specific Bean Origin_or Bar Name', 'REF',
       'Review_Date', 'Cocoa_Percent', 'Company_Location', 'Rating',
       'Bean_Type', 'Broad Bean_Origin'],
      dtype='object')

In [68]:
A = ['76%', '8.9%', '65%', '87.3%']
np.array([int(round(float(a.strip('%')),0)) for a in A])

array([76,  9, 65, 87])

In [8]:
y = df['Rating']
X = df.drop(['Rating', 'Bean_Type'], axis = 1)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_eval, X_test, y_eval, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [10]:
X_train.head()

,Company_(Maker-if known),Specific Bean Origin_or Bar Name,REF,Review_Date,Cocoa_Percent,Company_Location,Broad Bean_Origin
41,Alexandre,"La Dalia, Matagalpa",1944,2017,70%,Netherlands,Nicaragua
1728,Violet Sky,Cahabon,1502,2015,77%,U.S.A.,Guatemala
135,Artisan du Chocolat,Dominican Republic,363,2009,72%,U.K.,Dominican Republic
799,Heirloom Cacao Preservation (Zokoko),"Wild Beni, Lower Rio Beni, Tranquilidad, 2015",1744,2016,72%,U.S.A.,Bolivia
1611,Tabal,"Chiapas, Lacandon Jungle",1105,2013,70%,U.S.A.,Mexico


In [11]:
X_train['Cocoa_Percent'] = X_train['Cocoa_Percent'].apply(lambda x: float(x.strip('%')))
X_test['Cocoa_Percent'] = X_test['Cocoa_Percent'].apply(lambda x: float(x.strip('%')))

/home/anika/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
[len(X_train[column].unique()) for column in X_train.columns]

[342, 642, 418, 12, 37, 55, 77]

In [13]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelBinarizer

In [14]:
mapper = DataFrameMapper([
    ('Cocoa_Percent', None),
    ('Company_Location', LabelBinarizer()),
    ('Broad Bean_Origin', LabelBinarizer())
], df_out = True)

In [15]:
Z_train = mapper.fit_transform(X_train)
Z_test = mapper.transform(X_test)

## Naive model

rating goes up with cocoa percentage

In [16]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train[['Cocoa_Percent']], y_train)


# X_test['Cocoa_Percent'] = X_test['Cocoa_Percent'].apply(lambda x: float(x.strip('%')))
linreg.score(X_test[['Cocoa_Percent']], y_test)

0.00557433371423699

In [17]:
import matplotlib.pyplot as plt
plt.scatter(X_test[['Cocoa_Percent']], y_test);

## Feature Selection

In [18]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor


select = RFE(RandomForestRegressor(n_estimators=10, random_state=42), n_features_to_select=10)
select.fit(Z_train, y_train)
Z_train_selected = select.transform(Z_train)

In [19]:
Z_train.head()

,Cocoa_Percent,Company_Location_Amsterdam,Company_Location_Argentina,Company_Location_Australia,Company_Location_Austria,Company_Location_Belgium,Company_Location_Bolivia,Company_Location_Brazil,Company_Location_Canada,Company_Location_Chile,...,Broad Bean_Origin_Uganda,Broad Bean_Origin_Vanuatu,"Broad Bean_Origin_Ven., Indonesia, Ecuad.","Broad Bean_Origin_Ven., Trinidad, Mad.","Broad Bean_Origin_Venez,Africa,Brasil,Peru,Mex",Broad Bean_Origin_Venezuela,"Broad Bean_Origin_Venezuela, Carribean","Broad Bean_Origin_Venezuela, Dom. Rep.",Broad Bean_Origin_Vietnam,Broad Bean_Origin_West Africa
41,70.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1728,77.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135,72.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
799,72.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1611,70.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
select_df = pd.DataFrame(zip(Z_train.columns, select.support_), columns = ['feature', 'bool'])

In [21]:
select_df = select_df[select_df['bool']]

In [22]:
Z_train_selected = pd.DataFrame(Z_train_selected)
Z_train_selected.columns = select_df['feature'].values

In [23]:
Z_train_selected = Z_train_selected.astype(int)

In [24]:
linreg = LinearRegression()
linreg.fit(Z_train_selected, y_train)

Z_test_selected = Z_test[select_df['feature'].values]
linreg.score(Z_test_selected, y_test)

0.01630283302280977

In [25]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha = 0.01)

In [26]:
lasso.fit(Z_train_selected, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [27]:
lasso.score(Z_test_selected, y_test)

0.01099377595688722

In [28]:
lasso.coef_

array([-0.01444035, -0.        ,  0.        ,  0.        , -0.        ,
       -0.02995429, -0.        ,  0.        , -0.        ,  0.        ])

In [29]:
Z_train_selectedd = Z_train_selected[['Cocoa_Percent', 'Company_Location_U.S.A.']]
Z_test_selectedd = Z_test_selected[['Cocoa_Percent', 'Company_Location_U.S.A.']]

## CAtboost

In [30]:
# import os
import numpy as np
np.set_printoptions(precision=4)

# import catboost as cb

In [31]:
Z_train_selected.info()
Z_test_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 10 columns):
Cocoa_Percent                   1032 non-null int64
Company_Location_Ecuador        1032 non-null int64
Company_Location_France         1032 non-null int64
Company_Location_Italy          1032 non-null int64
Company_Location_U.K.           1032 non-null int64
Company_Location_U.S.A.         1032 non-null int64
Broad Bean_Origin_Ecuador       1032 non-null int64
Broad Bean_Origin_Madagascar    1032 non-null int64
Broad Bean_Origin_Peru          1032 non-null int64
Broad Bean_Origin_Venezuela     1032 non-null int64
dtypes: int64(10)
memory usage: 80.7 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 345 entries, 969 to 362
Data columns (total 10 columns):
Cocoa_Percent                   345 non-null float64
Company_Location_Ecuador        345 non-null int64
Company_Location_France         345 non-null int64
Company_Location_Italy          345 non-null int64
Company_Location_U.K

In [32]:
[print(x) for x in Z_train_selected[['Cocoa_Percent']]]

Cocoa_Percent


[None]

In [33]:
# Z_train_selected['Cocoa_percent'] = [print(x) for x in Z_train_selected['Cocoa_Percent']]
# Z_train_selected['Cocoa_Percent'] = Z_train_selected['Cocoa_Percent'].astype(int)
# Z_test_selected['Cocoa_percent'] = [round(x) for x in Z_test_selected['Cocoa_Percent']]
Z_test_selectedd['Cocoa_Percent'] = Z_test_selectedd['Cocoa_Percent'].astype(int)

/home/anika/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [34]:
Z_test_selected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 345 entries, 969 to 362
Data columns (total 10 columns):
Cocoa_Percent                   345 non-null float64
Company_Location_Ecuador        345 non-null int64
Company_Location_France         345 non-null int64
Company_Location_Italy          345 non-null int64
Company_Location_U.K.           345 non-null int64
Company_Location_U.S.A.         345 non-null int64
Broad Bean_Origin_Ecuador       345 non-null int64
Broad Bean_Origin_Madagascar    345 non-null int64
Broad Bean_Origin_Peru          345 non-null int64
Broad Bean_Origin_Venezuela     345 non-null int64
dtypes: float64(1), int64(9)
memory usage: 29.6 KB


In [35]:
cat_features = list(range(0, Z_train_selectedd.shape[1]))

In [36]:
from catboost import CatBoostRegressor, Pool

model = CatBoostRegressor(
    iterations=5,
    learning_rate=0.1
    #loss_function='Logloss',
    #loss_function='CrossEntropy'
)
train_pool = Pool(data=Z_train_selectedd, 
                  label=y_train, 
                  cat_features=cat_features)

validation_pool = Pool(data=Z_test_selectedd, 
                       label=y_test, 
                       cat_features=cat_features)
model.fit(
    train_pool,
    eval_set=validation_pool,
    verbose=False
)

In [37]:
model.score(Z_test_selectedd, y_test)

0.02423940787974277

## Gridsearch

In [38]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import recall_score, make_scorer, f1_score

In [39]:
model_params = {
    'iterations':[50, 100, 150],
    'learning_rate':[0.05, 0.1, 0.5],
}

model_gridsearch = GridSearchCV(CatBoostRegressor(), model_params, cv=5, verbose=1, n_jobs=-1)

model_gridsearch.fit(Z_train_selectedd, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


0:	learn: 0.4568883	total: 1.23ms	remaining: 60ms
1:	learn: 0.4538670	total: 2.27ms	remaining: 54.4ms
2:	learn: 0.4514697	total: 3.59ms	remaining: 56.3ms
3:	learn: 0.4502982	total: 4.32ms	remaining: 49.7ms
4:	learn: 0.4486119	total: 4.77ms	remaining: 42.9ms
5:	learn: 0.4475272	total: 5.16ms	remaining: 37.9ms
6:	learn: 0.4459566	total: 5.67ms	remaining: 34.8ms
7:	learn: 0.4449371	total: 6.09ms	remaining: 32ms
8:	learn: 0.4444661	total: 6.43ms	remaining: 29.3ms
9:	learn: 0.4434322	total: 7.25ms	remaining: 29ms
10:	learn: 0.4429935	total: 8.12ms	remaining: 28.8ms
11:	learn: 0.4420970	total: 9.97ms	remaining: 31.6ms
12:	learn: 0.4412331	total: 11.5ms	remaining: 32.6ms
13:	learn: 0.4409578	total: 11.9ms	remaining: 30.7ms
14:	learn: 0.4404112	total: 13.3ms	remaining: 31ms
15:	learn: 0.4398577	total: 14ms	remaining: 29.7ms
16:	learn: 0.4394511	total: 15ms	remaining: 29.1ms
17:	learn: 0.4389928	total: 16.2ms	remaining: 28.8ms
18:	learn: 0.4388152	total: 17ms	remaining: 27.7ms
19:	learn: 0.4386

[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    3.5s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=<catboost.core.CatBoostRegressor object at 0x7fcac8de8588>,
             iid='warn', n_jobs=-1,
             param_grid={'iterations': [50, 100, 150],
                         'learning_rate': [0.05, 0.1, 0.5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [40]:
model_gridsearch.best_score_

0.06578519759995281

In [41]:
model_gridsearch.best_params_

{'iterations': 50, 'learning_rate': 0.1}

## Pipe

ok so what does new info need to go through.
data comes in as us comapny and cocoa percent
then we need to transofrm the cocoa percent to strip the % and convert to int, label binarize the us company bool?
so mapper
then put through model?
final_model.predict
make it two digit with round(x,2)
get it out

In [42]:
from sklearn.pipeline import make_pipeline

In [49]:
def percent_stripper_int(x):
    return int(round(float(x.strip('%')),0))
percent_stripper_int('63.76%')
psi_func().fit_transform('63.76')

64

In [63]:
from sklearn.base import TransformerMixin
from sklearn.preprocessing import LabelEncoder
class psi_func(TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y= None):
        pass

    def transform(self, X, y= None):
        return int(round(float(X),0))

    def fit_transform(self, X, y= None):
        self.fit(X)
        return self.transform(X)


final_mapper = DataFrameMapper([
    ('Cocoa_Percent', psi_func()),
    ('USA_Company', None)
])


final_model = CatBoostRegressor(iterations = 50, learning_rate= 0.1)
final_model.fit(Z_train_selectedd, y_train)
pipe = make_pipeline(final_model)

0:	learn: 0.4568883	total: 7.03ms	remaining: 344ms
1:	learn: 0.4538670	total: 8.43ms	remaining: 202ms
2:	learn: 0.4514697	total: 9.41ms	remaining: 147ms
3:	learn: 0.4502982	total: 10.2ms	remaining: 117ms
4:	learn: 0.4486119	total: 12.2ms	remaining: 110ms
5:	learn: 0.4475272	total: 13.1ms	remaining: 96.4ms
6:	learn: 0.4459566	total: 15.7ms	remaining: 96.6ms
7:	learn: 0.4449371	total: 17.2ms	remaining: 90.1ms
8:	learn: 0.4444661	total: 19.1ms	remaining: 87.2ms
9:	learn: 0.4434322	total: 20.1ms	remaining: 80.5ms
10:	learn: 0.4429935	total: 21.2ms	remaining: 75.1ms
11:	learn: 0.4420970	total: 22.4ms	remaining: 71.1ms
12:	learn: 0.4412331	total: 23.4ms	remaining: 66.7ms
13:	learn: 0.4409578	total: 24.5ms	remaining: 63ms
14:	learn: 0.4404112	total: 26.6ms	remaining: 62.1ms
15:	learn: 0.4398577	total: 27.4ms	remaining: 58.1ms
16:	learn: 0.4394511	total: 27.9ms	remaining: 54.2ms
17:	learn: 0.4389928	total: 28.5ms	remaining: 50.6ms
18:	learn: 0.4388152	total: 28.8ms	remaining: 47.1ms
19:	learn:

In [57]:
pd.DataFrame({'Cocoa_Percent': '65.4',
              'Company_Location_U.S.A.': 1}, index = [0])

,Cocoa_Percent,Company_Location_U.S.A.
0,65.4,1


In [64]:
pipe.predict(pd.DataFrame({'Cocoa_Percent': 65,
                           'Company_Location_U.S.A.': 1}, index = [0]))

array([3.1337])

In [60]:
final_model = CatBoostRegressor(iterations = 50, learning_rate= 0.1)

In [ ]:
final_model.fit(Z_train_selectedd, y_train)

In [ ]:
final_model.score(Z_test_selectedd, y_test)

In [ ]:
two_digit_pred = [round(x,2) for x in final_model.predict(Z_test_selectedd).ravel()]

In [69]:
Z_test_selectedd.head()

,Cocoa_Percent,Company_Location_U.S.A.
969,65,1
905,72,1
441,80,0
104,70,1
691,69,1


## Alternate approach

In [ ]:
# len(X_alt)

In [ ]:
# alt_df = raw_df.copy()
# alt_df = alt_df.dropna(subset = ['Bean\nType','Broad Bean\nOrigin'])
# y_alt = alt_df['Rating']
# X_alt = alt_df['Bean\nType']

In [ ]:
# linreg = LinearRegression()
# lb = LabelBinarizer()

In [ ]:
# X_train_alt, X_test_alt, y_train_alt, y_test_alt = train_test_split(X_alt, y_alt, test_size=0.3, random_state=42)

In [ ]:
# Z_train_alt = lb.fit_transform(X_train_alt)
# Z_test_alt = lb.transform(X_test_alt)


# linreg.fit(Z_train_alt, y_train_alt)

In [ ]:
# linreg.score(Z_test_alt, y_test_alt)